In [1]:
import sys
sys.path.insert(0, "/Users/aryakakodkar/Documents/qerasure/python")
sys.path.insert(1, "/Users/aryakakodkar/Documents/qerasure/build")

In [2]:
from qerasure import RotatedSurfaceCode, NoiseParams

In [3]:
code = RotatedSurfaceCode(distance=5)

In [4]:
noise = NoiseParams(p_single_qubit_erasure=0.01)
noise.set("p_single_qubit_depolarize", 0.001)
noise.set("p_two_qubit_depolarize", 0.01)
noise.set("p_measurement_error", 0.005)

In [5]:
print(noise)

NoiseParams(p_single_qubit_depolarize=0.001000, p_single_qubit_erasure=0.010000, p_two_qubit_depolarize=0.010000, p_measurement_error=0.005000, p_two_qubit_erasure=0.000000, p_erasure_check_error=0.000000)
